In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import VQC
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np

import matplotlib.pyplot as plt
import time

2022-09-25 18:02:27.656788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-25 18:02:27.656836: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-25 18:02:32.514819: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-25 18:02:32.514857: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-09-25 18:02:32.515219: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [3]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.pca = 4
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 64
args.validation_split = 0.05
args.hinge_labels = 1
args.use_quantum = 1

# Base Model
args.learning_rate = 1e-1
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.cluster_state = 1
args.ansatz = 'NQubit'

In [4]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Performing PCA on data...
Cumulative sum on train : 55.622551252777555
Cumulative sum on test: 55.622551252777555
Standardizing data...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 2, 2, 1) │ (634, 2, 2, 1) │ (2115, 2, 2, 1) │ (2, 2, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031,)         │ (634,)         │ (2115,)         │ (1,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═════

In [13]:
data.config()

{'input_dims': (2, 2, 1), 'output_dims': (1,), 'mapping': [0, 1]}

In [14]:
model = VQC(data.config(), args)

In [15]:
model.compile()
model.build_graph().summary()

Model: "VQC-AngleMap-NQubit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2, 2, 1)]         0         
                                                                 
 NQubitPQC (NQubitPQC)       (None, 1)                 15        
                                                                 
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [17]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

Epoch 1/10
188/188 [==============================] - 10s 50ms/step - loss: 0.3970 - q_auc_2: 0.9597 - custom_accuracy: 0.8816 - val_loss: 0.3462 - val_q_auc_2: 0.9801 - val_custom_accuracy: 0.9119 - lr: 0.1000
Epoch 2/10
188/188 [==============================] - 9s 50ms/step - loss: 0.3442 - q_auc_2: 0.9811 - custom_accuracy: 0.9072 - val_loss: 0.3831 - val_q_auc_2: 0.9715 - val_custom_accuracy: 0.8836 - lr: 0.1000
Epoch 3/10
188/188 [==============================] - 9s 50ms/step - loss: 0.3638 - q_auc_2: 0.9755 - custom_accuracy: 0.8992 - val_loss: 0.3271 - val_q_auc_2: 0.9868 - val_custom_accuracy: 0.9353 - lr: 0.1000
Epoch 4/10
188/188 [==============================] - 10s 53ms/step - loss: 0.3437 - q_auc_2: 0.9815 - custom_accuracy: 0.9047 - val_loss: 0.3481 - val_q_auc_2: 0.9831 - val_custom_accuracy: 0.9139 - lr: 0.1000
Epoch 5/10
188/188 [==============================] - 10s 51ms/step - loss: 0.3467 - q_auc_2: 0.9812 - custom_accuracy: 0.9075 - val_loss: 0.3464 - val_q_auc_

In [18]:
model.test(data, callbacks=callbacks)

34/34 [==============================] - 1s 33ms/step - loss: 0.4212 - q_auc_2: 0.9634 - custom_accuracy: 0.8845


[0.42117348313331604,
 0.9634190201759338,
 0.8844975233078003,
 0.9634190201759338]

In [19]:
args.sparse = True

model = VQC(data.config(), args)
model.compile()
model.build_graph().summary()

Model: "VQC-AngleMap-NQubit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 2, 2, 1)]         0         
                                                                 
 NQubitPQC (NQubitPQC)       (None, 1)                 15        
                                                                 
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(data, callbacks)

Epoch 1/10
188/188 [==============================] - 9s 46ms/step - loss: 0.7416 - q_auc_3: 0.8168 - custom_accuracy: 0.7245 - val_loss: 0.7296 - val_q_auc_3: 0.8306 - val_custom_accuracy: 0.7323 - lr: 0.1000
Epoch 2/10
188/188 [==============================] - 9s 45ms/step - loss: 0.6883 - q_auc_3: 0.8659 - custom_accuracy: 0.7429 - val_loss: 0.6176 - val_q_auc_3: 0.9223 - val_custom_accuracy: 0.7666 - lr: 0.1000
Epoch 3/10
188/188 [==============================] - 10s 53ms/step - loss: 0.5968 - q_auc_3: 0.9114 - custom_accuracy: 0.7777 - val_loss: 0.3139 - val_q_auc_3: 0.9953 - val_custom_accuracy: 0.8926 - lr: 0.1000
Epoch 4/10
188/188 [==============================] - 10s 52ms/step - loss: 0.2916 - q_auc_3: 0.9895 - custom_accuracy: 0.9076 - val_loss: 0.2922 - val_q_auc_3: 0.9934 - val_custom_accuracy: 0.8850 - lr: 0.1000
Epoch 5/10
188/188 [==============================] - 10s 54ms/step - loss: 0.2813 - q_auc_3: 0.9907 - custom_accuracy: 0.9042 - val_loss: 0.2604 - val_q_auc_

In [21]:
model.test(data, callbacks=callbacks)

34/34 [==============================] - 1s 33ms/step - loss: 0.2474 - q_auc_3: 0.9964 - custom_accuracy: 0.9236


[0.2474074363708496,
 0.9964429140090942,
 0.9235600233078003,
 0.9964429140090942]